In [12]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.init as init
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [13]:
cifar10_train = datasets.CIFAR10(root='data/cifar10_data',
                                      train=True,
                                      download=True,
                                      transform=transforms.Compose([transforms.ToTensor()]))
cifar10_test = datasets.CIFAR10(root='data/cifar10_data',
                                      train=False,
                                      download=True,
                                      transform=transforms.Compose([transforms.ToTensor()]))

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(0)
if device =='cuda':
  torch.cuda.manual_seed_all(0)

In [15]:
import numpy as np
from copy import deepcopy

learning_rate = 0.001
epochs = 10
batch_size = 100

lowest_loss = np.inf
lowest_epoch = np.inf
early_stop = 5

In [16]:
train_loader = DataLoader(dataset=cifar10_train,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)
test_loader = DataLoader(dataset=cifar10_test,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)

In [19]:
for x,y in train_loader:
  print(x.size(), y.size())
  break

torch.Size([100, 3, 32, 32]) torch.Size([100])


In [22]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.layer1 = nn.Sequential(
      nn.Conv2d(3,32,3,1,1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
    )
    # image shape(100, 3, 32, 32)
    # conv1 -> (3, 32, 32, 32)
    # pool -> (3, 32, 16, 16)

    self.layer2 = nn.Sequential(
      nn.Conv2d(32,64,3,1,1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
    )
    # image shape(?, 32, 16, 16)
    # conv1 -> (32, 64, 16, 16)
    # pool -> (32, 64, 8, 8)

    self.layer3 = nn.Sequential(
      nn.Conv2d(64,128,3,1,1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
    )

    self.fc = nn.Sequential(
      nn.Linear(128*4*4, 100, bias=True),
      nn.LeakyReLU(),
      nn.Linear(100,10,bias=True)
    )
    # nn.Linear(128*4*4, 100, bias=True)
    # nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0),-1)
    out = self.fc(out)
    return out


In [23]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
total_batch = len(train_loader)
test_batch = len(test_loader)

In [25]:
for epoch in range(epochs):
  avg_cost = 0

  for x, y in train_loader:
    x = x.to(device)
    y = y.to(device)

    optimizer.zero_grad()
    y_hat = model(x)
    cost = criterion(y_hat, y)
    cost.backward()
    optimizer.step()

    avg_cost += cost
  avg_cost = avg_cost/total_batch
  print(f'epoch:{epoch}, cost:{avg_cost}')

epoch:0, cost:1.5736898183822632
epoch:1, cost:1.1620349884033203
epoch:2, cost:0.9536198973655701
epoch:3, cost:0.8349772691726685
epoch:4, cost:0.7394669651985168
epoch:5, cost:0.6658084988594055
epoch:6, cost:0.596147894859314
epoch:7, cost:0.5354154109954834
epoch:8, cost:0.4830319285392761
epoch:9, cost:0.42939481139183044


In [26]:
with torch.no_grad():
  avg_accuracy = 0

  for x,y in test_loader:
    x = x.to(device)
    y = y.to(device)
    pred = model(x)
    correct_pred = torch.argmax(pred, -1) == y
    accuracy = correct_pred.float().sum()
    avg_accuracy += accuracy
  avg_accuracy = avg_accuracy/test_batch

print(f'Accuracy: {avg_accuracy}')

Accuracy: 75.31999969482422
